#Data Analysis

In [2]:
import pandas as pd
import math
from datetime import datetime

# bibliotecas para padronizar os dados e buscar pelas variaveis mais representativas
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE

# separacao em treino/teste e modelo de regressao
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

# metricas
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score

In [3]:
#utilizei o meu drive pessoal para carregar os dados propostos
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#lendo os arquivos CSV
df = pd.read_csv('/content/drive/MyDrive/Processos Seletivos/transactional-sample.csv')

###Column Meanings
  - ransaction_id: identification of the transaction
  - merchant_id: identification of the merchant
  - user_id: identification of the user
  - card_number: number of the card used in the transaction
  - transaction_date: when the transaction took place
  - transaction_amount: the amount of the transaction
  - device_id: the identification of the device used in the transaction (some of them are missing)
  - has_cbk: tell us if chargeback happened or not.

In [5]:
df.head()

,transaction_id,merchant_id,user_id,card_number,transaction_date,transaction_amount,device_id,has_cbk
0,21320398,29744,97051,434505******9116,2019-12-01T23:16:32.812632,374.56,285475.0,False
1,21320399,92895,2708,444456******4210,2019-12-01T22:45:37.873639,734.87,497105.0,True
2,21320400,47759,14777,425850******7024,2019-12-01T22:22:43.021495,760.36,NaN,False
3,21320401,68657,69758,464296******3991,2019-12-01T21:59:19.797129,2556.13,NaN,True
4,21320402,54075,64367,650487******6116,2019-12-01T21:30:53.347051,55.36,860232.0,False


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3199 entries, 0 to 3198
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   transaction_id      3199 non-null   int64  
 1   merchant_id         3199 non-null   int64  
 2   user_id             3199 non-null   int64  
 3   card_number         3199 non-null   object 
 4   transaction_date    3199 non-null   object 
 5   transaction_amount  3199 non-null   float64
 6   device_id           2369 non-null   float64
 7   has_cbk             3199 non-null   bool   
dtypes: bool(1), float64(2), int64(3), object(2)
memory usage: 178.2+ KB


In [7]:
df[df.device_id.isnull()].has_cbk.describe()

count       830
unique        2
top       False
freq        763
Name: has_cbk, dtype: object

In [8]:
print("Formato:", df.shape)
print("\n\n\n")
for coluna in df.columns:
  print(coluna + ": ")
  print(df[coluna].unique())
  print(df[coluna].describe())
  display (df[df[coluna].isnull()])
  print("\n\n\n")

Formato: (3199, 8)




transaction_id: 
[21320398 21320399 21320400 ... 21323594 21323595 21323596]
count    3.199000e+03
mean     2.132200e+07
std      9.236161e+02
min      2.132040e+07
25%      2.132120e+07
50%      2.132200e+07
75%      2.132280e+07
max      2.132360e+07
Name: transaction_id, dtype: float64


,transaction_id,merchant_id,user_id,card_number,transaction_date,transaction_amount,device_id,has_cbk






merchant_id: 
[29744 92895 47759 ...   656 52493  9603]
count     3199.000000
mean     48771.128790
std      29100.360839
min         16.000000
25%      23426.000000
50%      48752.000000
75%      73915.000000
max      99799.000000
Name: merchant_id, dtype: float64


,transaction_id,merchant_id,user_id,card_number,transaction_date,transaction_amount,device_id,has_cbk






user_id: 
[97051  2708 14777 ... 59275     7     8]
count     3199.000000
mean     50891.077212
std      29515.282827
min          6.000000
25%      24267.500000
50%      52307.000000
75%      76837.000000
max      99974.000000
Name: user_id, dtype: float64


...

,transaction_id,merchant_id,user_id,card_number,transaction_date,transaction_amount,device_id,has_cbk






device_id: 
[2.85475e+05 4.97105e+05         nan ... 2.00000e+00 6.11488e+05
 4.00000e+00]
count      2369.000000
mean     493924.859856
std      283785.584545
min           2.000000
25%      259344.000000
50%      495443.000000
75%      733243.000000
max      999843.000000
Name: device_id, dtype: float64


,transaction_id,merchant_id,user_id,card_number,transaction_date,transaction_amount,device_id,has_cbk
2,21320400,47759,14777,425850******7024,2019-12-01T22:22:43.021495,760.36,NaN,False
3,21320401,68657,69758,464296******3991,2019-12-01T21:59:19.797129,2556.13,NaN,True
18,21320416,97583,6434,498401******3796,2019-12-01T19:56:26.207496,396.84,NaN,False
21,21320419,3788,18009,550209******5149,2019-12-01T19:44:40.414039,296.27,NaN,False
32,21320430,56977,69758,464296******3991,2019-12-01T19:17:21.731168,2803.32,NaN,True
...,...,...,...,...,...,...,...,...
3194,21323592,50493,49581,650486******4139,2019-11-01T13:05:34.054967,744.15,NaN,False
3195,21323593,9603,59275,528052******3611,2019-11-01T11:47:02.404963,1403.67,NaN,False
3196,21323594,57997,84486,522688******9874,2019-11-01T10:23:50.555604,1.55,NaN,False
3197,21323595,35930,7,544315******7773,2019-11-01T01:29:45.799767,359.68,NaN,False






has_cbk: 
[False  True]
count      3199
unique        2
top       False
freq       2808
Name: has_cbk, dtype: object


,transaction_id,merchant_id,user_id,card_number,transaction_date,transaction_amount,device_id,has_cbk


In [9]:
#replacing null values for number '0'
df.device_id = df.device_id.fillna(0)

In [10]:
#sorting out the 'transaction_date_dt'
df['transaction_date_dt'] = pd.to_datetime(df['transaction_date'], errors='coerce')    #corrige os erros com os valores das datas de nascimento, adequa para DateTime
df['day'] = df['transaction_date_dt'].dt.day
df['month'] = df['transaction_date_dt'].dt.month
df['year'] = df['transaction_date_dt'].dt.year
df['time'] = df['transaction_date_dt'].dt.time
df['hour'] = df['transaction_date_dt'].dt.hour

#ML models

In [154]:
#sorting out the target data ans other columns to test the model
X = df.drop(["has_cbk","transaction_date","transaction_date_dt","day","month","year","time"], axis=1)
y = df.has_cbk

In [12]:
#replacing '*' ind the card_numbers for none to use in the model, we need just numbers
X.card_number = X.card_number.str.translate({ord(i): None for i in '*'})

In [13]:
dic_has_cbk = {False: 0, True: 1}
y = y.map(dic_has_cbk)

In [14]:
#normalizing X
X_std = StandardScaler().fit_transform(X)
#sorting out the training and test data in 20%
X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size = 0.2, random_state=10) 


In [15]:
#fist model - DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

df_confusion_matrix = pd.DataFrame(confusion_matrix(y_test, y_pred), 
            columns=["Predicted False","Predicted True"],
            index  =["Predicted False", "Predicted True"])
df_confusion_matrix

,Predicted False,Predicted True
Predicted False,510,47
Predicted True,33,50


In [16]:
metrics = pd.DataFrame({"Metrics":["Accuracy","Precision","Specificity","Sensitivity"]})

def finals_metrics(df,m):
  ac = accuracy_score(y_test, y_pred)
  pre = precision_score(y_test, y_pred)
  esp = df_confusion_matrix.iloc[0,0] / (df_confusion_matrix.iloc[0,1] + df_confusion_matrix.iloc[0,0])
  sens = df_confusion_matrix.iloc[1,1] / (df_confusion_matrix.iloc[1,1] + df_confusion_matrix.iloc[1,0])
  metrics[m] = [ac,pre,esp,sens]
  return metrics

finals_metrics(df,'DecisionTree')

,Metrics,DecisionTree
0,Accuracy,0.875000
1,Precision,0.515464
2,Specificity,0.915619
3,Sensitivity,0.602410


In [17]:
##second model - LogisticRegression
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

df_confusion_matrix = pd.DataFrame(confusion_matrix(y_test, y_pred), 
            columns=["Predicted False","Predicted True"],
            index  =["Predicted False", "Predicted True"])
df_confusion_matrix

,Predicted False,Predicted True
Predicted False,552,5
Predicted True,75,8


In [18]:
finals_metrics(df,'LogisticRegression')

,Metrics,DecisionTree,LogisticRegression
0,Accuracy,0.875000,0.875000
1,Precision,0.515464,0.615385
2,Specificity,0.915619,0.991023
3,Sensitivity,0.602410,0.096386


In [19]:
##third model - GaussianNaiveBayes
gnb = GaussianNB()
lr.fit(X_train, y_train)
y_pred = gnb.fit(X_train, y_train).predict(X_test)

df_confusion_matrix = pd.DataFrame(confusion_matrix(y_test, y_pred), 
            columns=["Predicted False","Predicted True"],
            index  =["Predicted False", "Predicted True"])
df_confusion_matrix

,Predicted False,Predicted True
Predicted False,532,25
Predicted True,69,14


In [20]:
finals_metrics(df,'GaussianNB')

,Metrics,DecisionTree,LogisticRegression,GaussianNB
0,Accuracy,0.875000,0.875000,0.853125
1,Precision,0.515464,0.615385,0.358974
2,Specificity,0.915619,0.991023,0.955117
3,Sensitivity,0.602410,0.096386,0.168675


In [ ]:
##second model - DecisionTreeClassifier
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [21]:
#testing with Transaction_id
def transaction_id_pred(id):
   teste = X[X.transaction_id == id]
   X_teste = StandardScaler().fit_transform(teste)
   y_pred = lr.predict(X_teste)
   if y_pred == 0:
    return print('Transaction_id:',id,'\nRecommendation:' , 'approve')
   else :
    return print('Transaction_id:',id,'\nRecommendation:' , 'repprove')


In [22]:
transaction_id_pred(21320398)

Transaction_id: 21320398 
Recommendation: approve


#Anti-Fraud Method

In [164]:
#criando uma transação para teste
transacao_teste = pd.DataFrame({"transaction_id": [21323597], "merchant_id":[56107] , "user_id":[81152], "card_number":["650516******9201"], "transaction_date": ["2019-12-01T21:04:55.066909"], 
                                "transaction_amount":[345.68], "device_id":[486.0], "has_cbk": [""] })

#sorting out the 'transaction_date_dt'
transacao_teste['transaction_date_dt'] = pd.to_datetime(transacao_teste['transaction_date'], errors='coerce')    #corrige os erros com os valores das datas de nascimento, adequa para DateTime
transacao_teste['day'] = transacao_teste['transaction_date_dt'].dt.day
transacao_teste['month'] = transacao_teste['transaction_date_dt'].dt.month
transacao_teste['year'] = transacao_teste['transaction_date_dt'].dt.year
transacao_teste['time'] = transacao_teste['transaction_date_dt'].dt.time
transacao_teste['hour'] = transacao_teste['transaction_date_dt'].dt.hour

transacao_teste

,transaction_id,merchant_id,user_id,card_number,transaction_date,transaction_amount,device_id,has_cbk,transaction_date_dt,day,month,year,time,hour
0,21323597,56107,81152,650516******9201,2019-12-01T21:04:55.066909,345.68,486.0,,2019-12-01 21:04:55.066909,1,12,2019,21:04:55.066909,21


In [156]:
X_transacao_teste = transacao_teste.drop(["has_cbk","transaction_date","transaction_date_dt","day","month","year","time"], axis=1)

In [160]:
def testes(transacao_teste):
 
  #Reject transactions above a certain amount in a given period;
  if ((transacao_teste.hour[0] <= 7) or (transacao_teste.hour[0] >= 21)) and (transacao_teste.transaction_amount[0] >= 1000):
      return print('Transaction_id:',id,'\nRecommendation:' , 'repprove due to time and high amount')
      transacao_teste.has_cbk[0] = True
  
  #Reject transaction if user is trying too many transactions in a row;
  t_id_suspeita_user = []
  t_id_suspeita_card = []
  soma_valor_user = 0
  soma_valor_card = 0
  
  for i in range(len(df)):
      #Reject transaction if a user had a chargeback before (note that this information does not comes on the payload. The chargeback data is received days after the transaction was approved)
      if (transacao_teste.user_id[0] == df.user_id[i]) and (df.has_cbk == True):
          return print('Transaction_id:',id,'\nRecommendation:' , 'repprove due to time and high amount')
          transacao_teste.has_cbk[0] = True
          break
      # transaction with same user_id and time less then 1000 seconds = 16,6 min
      if (transacao_teste.user_id[0] == df.user_id[i]) and (math.fabs((transacao_teste.transaction_date_dt[0] - df.transaction_date_dt[i]).total_seconds()) <  1000):
          t_id_suspeita_user.append(i)
          #check if have more than 3 transaction for the same user_id and a short time period
          if  len(t_id_suspeita_user) > 3:
              for j in range(len(t_id_suspeita_user)):
                  soma_valor_user += df.transaction_amount[t_id_suspeita_user[j]]
              soma_valor_user += transacao_teste.transaction_amount[0]
              #check if the amount is more then 2000, so reprove the transaction
              if soma_valor_user > 2000:
                  return print('Transaction_id:',id,'\nRecommendation:' , 'repprove for user_id repeat')        
                  transacao_teste.has_cbk[0] = True
                  #if is possible we can chargebacks all this transactions
                  break
      # transaction with same card_number and time less then 1000 seconds = 16,6 min
      if (transacao_teste.card_number[0] == df.card_number[i]) and (math.fabs((transacao_teste.transaction_date_dt[0] - df.transaction_date_dt[i]).total_seconds()) <  1000):
          #check if have more than 3 transaction for the same user_id and a short time period
          t_id_suspeita_card.append(i)
          if  len(t_id_suspeita_card) > 3:
              for j in range(len(t_id_suspeita_card)):
                  soma_valor_card += df.transaction_amount[t_id_suspeita_card[j]]
              soma_valor_card += transacao_teste.transaction_amount[0]
              #check if the amount is more then 2000, so reprove the transaction
              if soma_valor_card > 2000:
                  return print('Transaction_id:',id,'\nRecommendation:' , 'repprove for user_id repeat')        
                  transacao_teste.has_cbk[0] = True
                  #if is possible we can chargebacks all this transactions
                  break

  #using Machine Learning
  X_transacao_teste = transacao_teste.drop(["has_cbk","transaction_date","transaction_date_dt","day","month","year","time"], axis=1)
  X_transacao_teste.card_number = X_transacao_teste.card_number.str.translate({ord(i): None for i in '*'})
  X_transacao_teste = StandardScaler().fit_transform(X_transacao_teste)
  if y_pred == 1:
      return print('Transaction_id:',id,'\nRecommendation:' , 'repprove ML model')  
      transacao_teste.has_cbk[0] = True


  if transacao_teste.has_cbk[0] != True:
      transacao_teste.has_cbk = False
      return print('Transaction_id:',id,'\nRecommendation:' , 'approve') 


In [161]:
testes(transacao_teste)